In [1]:
import random

random.seed(0)

# num nodes
N = 9

data = []
for i in range(N - 1):
    for j in range(i + 1, N):
        if random.random() < 0.5:
            w = random.randint(1, 20) - 10
            w = w if w !=0 else 1
            data.append([i, j, w])

# num edges
M = len(data)

print("num nodes : ", N)
print("num edges : ", M)
print(data)

num nodes :  9
num edges :  14
[[0, 3, -1], [0, 5, 1], [0, 7, -3], [1, 2, -6], [1, 4, 8], [1, 8, -7], [2, 5, -6], [2, 6, 1], [3, 5, 7], [3, 6, 8], [3, 8, 3], [5, 7, 1], [6, 7, 9], [6, 8, -5]]


In [2]:
import sys

class Node:
    def __init__(self, idx):
        self.idx = idx
        self.used = False
        self.dist = sys.maxsize
    
    def __repr__(self):
        return f'{self.idx}/T' if self.used else f'{self.idx}/F'

    def init_node(self):
        self.used = False
        self.prev = None
        self.dist = sys.maxsize


class Edge:
    def __init__(self, nds, nde, w):
        self.s = nds
        self.e = nde
        self.w = w
    
    def __repr__(self):
        return f"{self.s} , {self.e} ({self.w})"

    
class Graph:
    def __init__(self, N, data):
        if N <= 0:
            raise ValueError("graph must contain at least 1 node")

        self.num_nodes = N
        self.nodes = [Node(idx) for idx in range(N)]
        self.edges = [Edge(self.nodes[i], self.nodes[j], w) for i, j, w in data]
        self.graph = [[] for _ in range(N)]
        self.build_graph()
    
    def __len__(self):
        return self.num_nodes
    
    def __repr__(self):
        str_ = ""
        for edges in self.graph:
            if len(edges) == 0:
                continue
            str_ += "{} -> [".format(edges[0].s)
            for edge in edges:
                str_ += " {} ({}),".format(edge.e, edge.w)
            str_ =  str_ + "]\n"
        return str_[:-1]

    def build_graph(self):
        for edge in self.edges:
            self.graph[edge.s.idx].append(edge)

    def init_graph(self):
        for nd in self.nodes:
            nd.init_node()
    
    def show_nodes(self):
        for nd in self.nodes:
            print(nd)

    def get_edges(self, nd):
        return self.graph[nd.idx]


In [3]:
def floyd_warshall(G, N):
    G.init_graph()
    
    dp = [[float('INF') for i in range(N)] for j in range(N)]
    md = [[-2 for i in range(N)] for j in range(N)]
    
    for i in range(N):
        dp[i][i] = 0
        md[i][i] = i
    
    for edge in G.edges:
        dp[edge.s.idx][edge.e.idx] = edge.w
        md[edge.s.idx][edge.e.idx] = -1
    
    for k in range(N):
        for i in range(N):
            for j in range(N):
                if dp[i][j] > dp[i][k] + dp[k][j]:
                    print(i, j, k)
                    print(dp[i][j], dp[i][k], dp[k][j])
                    print()
                    dp[i][j] = dp[i][k] + dp[k][j]
                    md[i][j] = k

    return dp, md

def get_path(md, s, e):
    if md[s][e] == -2:
        return "not connected"
    if md[s][e] == -1:
        return [s, e]

    k = md[s][e]
    sk = get_path(md, s, k)
    ke = get_path(md, k, e)
    
    return sk + ke[1:]

In [4]:
float('INF')

inf

In [5]:
G = Graph(N, data)
print(G)

0/F -> [ 3/F (-1), 5/F (1), 7/F (-3),]
1/F -> [ 2/F (-6), 4/F (8), 8/F (-7),]
2/F -> [ 5/F (-6), 6/F (1),]
3/F -> [ 5/F (7), 6/F (8), 8/F (3),]
5/F -> [ 7/F (1),]
6/F -> [ 7/F (9), 8/F (-5),]


In [6]:
dp, md = floyd_warshall(G, N)

1 5 2
inf -6 -6

1 6 2
inf -6 1

0 6 3
inf -1 8

0 8 3
inf -1 3

1 7 5
inf -12 1

2 7 5
inf -6 1

3 7 5
inf 7 1

1 8 6
-7 -5 -5

2 8 6
inf 1 -5



In [7]:
dp

[[0, inf, inf, -1, inf, 1, 7, -3, 2],
 [inf, 0, -6, inf, 8, -12, -5, -11, -10],
 [inf, inf, 0, inf, inf, -6, 1, -5, -4],
 [inf, inf, inf, 0, inf, 7, 8, 8, 3],
 [inf, inf, inf, inf, 0, inf, inf, inf, inf],
 [inf, inf, inf, inf, inf, 0, inf, 1, inf],
 [inf, inf, inf, inf, inf, inf, 0, 9, -5],
 [inf, inf, inf, inf, inf, inf, inf, 0, inf],
 [inf, inf, inf, inf, inf, inf, inf, inf, 0]]

In [8]:
md

[[0, -2, -2, -1, -2, -1, 3, -1, 3],
 [-2, 1, -1, -2, -1, 2, 2, 5, 6],
 [-2, -2, 2, -2, -2, -1, -1, 5, 6],
 [-2, -2, -2, 3, -2, -1, -1, 5, -1],
 [-2, -2, -2, -2, 4, -2, -2, -2, -2],
 [-2, -2, -2, -2, -2, 5, -2, -1, -2],
 [-2, -2, -2, -2, -2, -2, 6, -1, -1],
 [-2, -2, -2, -2, -2, -2, -2, 7, -2],
 [-2, -2, -2, -2, -2, -2, -2, -2, 8]]

In [9]:
s = 0
e = 8
path = get_path(md, s, e)
print(path, dp[s][e])

[0, 3, 8] 2


In [10]:
G.graph[0][0]

0/F , 3/F (-1)

In [11]:
G.graph[3][2]

3/F , 8/F (3)